In [2]:
import pandas as pd
import altair as alt

# Cargar datos
source = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')

# Formatear fechas y eliminar cancelados
source['Fecha'] = pd.to_datetime(source['Fecha'], format='%d-%m-%Y', errors='coerce')
source = source[source['Cancelado'] != 1]

# Clasificación
def clasificar_familia(genero):
    g = str(genero).lower().strip()
    if g in ['reggaeton', 'trap', 'rap', 'hip hop', 'urbana']:
        return 'Urbano'
    elif g in ['rock', 'rock & roll', 'metal', 'heavy metal', 'punk']:
        return 'Rock'
    elif g in ['pop', 'pop balada', 'pop rock', 'trippy pop', 'k-pop']:
        return 'Pop'
    elif g in ['balada']:
        return 'Balada'
    elif g in ['electronica', 'disco']:
        return 'Electrónica'
    elif g in ['salsa', 'cumbia', 'mariachi', 'ranchera', 'vallenato', 'rumba', 'ska', 'reggae']:
        return 'Tropical/Latino'
    else:
        return 'Otros'

# Aplicar clasificación
source['Familia_musical'] = source['Genero_musical'].apply(clasificar_familia)

# Quitar duplicados para que cada artista cuente una sola vez por familia musical
artistas_unicos = source[['Artista', 'Familia_musical']].drop_duplicates()

# Contar artistas únicos por familia musical
conteo_familias = artistas_unicos.groupby('Familia_musical').size().reset_index(name='Cantidad')

# Ordenar con "Otros" al final
conteo_familias['orden'] = conteo_familias['Familia_musical'].apply(lambda x: 99 if x == 'Otros' else 0)
conteo_familias = conteo_familias.sort_values(['orden', 'Cantidad'], ascending=[True, False]).drop(columns='orden')

colores = ['#080D21', '#46007E', '#3F4270', '#A20264', '#A34EA0', '#FF05BB', '#F4F0CD']

# Crear un diccionario con las familias y los géneros que agrupan
familias_generos = {
    'Urbano': ['reggaeton', 'trap', 'rap', 'hip hop', 'urbana'],
    'Rock': ['rock', 'rock & roll', 'metal', 'heavy metal', 'punk'],
    'Pop': ['pop', 'pop balada', 'pop rock', 'trippy pop', 'k-pop'],
    'Balada': ['balada'],
    'Electrónica': ['electronica', 'disco'],
    'Tropical/Latino': ['salsa', 'cumbia', 'mariachi', 'ranchera', 'vallenato', 'rumba', 'ska', 'reggae', 'folclore chileno'],
    'Otros': ['r&b', 'soul','clasica','cuarteto', 'instrumental',  'infantil']
}

# Añadir columna con géneros como string separados por coma
conteo_familias['Géneros agrupados'] = conteo_familias['Familia_musical'].map(
    lambda f: ', '.join(familias_generos.get(f, []))
)
# Ordenar conteo_familias por cantidad descendente
conteo_familias = conteo_familias.sort_values('Cantidad', ascending=False).reset_index(drop=True)

# Obtener el orden de las familias musicales (según cantidad)
orden_familias = conteo_familias['Familia_musical'].tolist()

# Asignar colores según ese orden (considerando que tienes tantos colores como familias)
colores_ordenados = ['#0A1A40', '#5B1FA8', '#47528B', '#B0317B', '#C063BA', '#FF33CC', '#F4F0CD']
# Si hay menos familias que colores, corta la lista; si hay más, puedes ajustar o repetir
colores_asignados = colores_ordenados[:len(orden_familias)]

# Mapear familia a color
color_domain = orden_familias
color_range = colores_asignados

chart = alt.Chart(conteo_familias).mark_bar().encode(
    y=alt.Y('Familia_musical:N', sort=orden_familias, title='Familia musical'),
    x=alt.X('Cantidad:Q', title='Cantidad de artistas únicos'),
    color=alt.Color('Familia_musical:N',
                    scale=alt.Scale(domain=color_domain, range=color_range),
                    legend=None),
    tooltip=['Familia_musical', 'Cantidad', 'Géneros agrupados']
).properties(
    width=600,
    height=300,
    title='Artistas únicos por familia musical'
)
chart

alt.Chart(...)